In [1]:
%load_ext autoreload
%autoreload 2

# Preamble

In [ ]:
import pandas as pd
import numpy as np


import os

from pathlib import Path

from modelling_pkg.config import *
from modelling_pkg.mappings import *

from tqdm.notebook import tqdm

pd.set_option("display.max_columns", None)

# ORGANIZATION_ID = get_org_id('data')

ORGANIZATIONS, ORGANIZATION_ID = get_organizations(DATADIR)

ORG_ID_R = {v: k for k, v in ORGANIZATION_ID.items()}

In [ ]:
dual_orgs = ORGANIZATIONS[ORGANIZATIONS.dual_currency].copy()
dual_org_country_names = list(dual_orgs.index)
print(dual_org_country_names)
# display(dual_orgs)
ctr2ex_rate_dom2int = dual_orgs.ex_rate_dom2int
ctr2ex_rate_dom2int

# Data import

In [ ]:
countries_dir = DATADIR + "/countries"

Path(countries_dir).mkdir(parents=True, exist_ok=True)
print(countries_dir)

In [ ]:
%%time
offer_, audit_, oppo_, customer_, shipment_ = {}, {}, {}, {}, {}
for i, org_id in enumerate(ORG_ID_R.keys()):
    print(i, org_id)
    offer_[org_id] = pd.read_csv(
        f"{DATADIR}/raw_tables/OFFERS/{org_id}.csv",
        parse_dates=[
            "creation_date",
            "updated_date",
            "offer_expiry_date",
            "effective_start_date",
            "agreement_renewal_date",
        ],
    )
    audit_[org_id] = pd.read_csv(
        f"{DATADIR}/raw_tables/OFFER_AUDIT/{org_id}.csv",
        parse_dates=["created_date", "updated_date"],
    )
    oppo_[org_id] = pd.read_csv(
        f"{DATADIR}/raw_tables/OPPORTUNITY/{org_id}.csv",
        parse_dates=[
            "created_date",
            "actual_close_date",
            "expected_close_date",
            "pipeline_last_updated_date",
        ],
    )
    customer_[org_id] = pd.read_csv(
        f"{DATADIR}/raw_tables/CUSTOMER/{org_id}.csv",
        parse_dates=["created_date", "ftb_month"],
    )
    shipment_[org_id] = pd.read_csv(
        f"{DATADIR}/raw_tables/SHIPMENT_PROFILE/{org_id}.csv",
        parse_dates=["created_date", "last_updated_date"],
    )

In [10]:
counts = defaultdict(list)

In [ ]:
print(
    offer_["DHL Italy"].creation_date.min().date(),
    offer_["DHL Italy"].creation_date.max().date(),
)

In [20]:
# ctrs = sorted([ORG_ID_R[ctr_id] for ctr_id in offer_.organization_id.unique() if ctr_id in ORG_ID_R])
# ctrs = ['COLUMBIA']
# print(ctrs)
# ctrs = [ctr for ctr in ['South_Africa', 'Nigeria', 'Mexico', 'Canada', 'United_States', 'Kenya'] if ctr in ctrs]
# ctrs

In [18]:
# # onb = ['Indonesia', 'Bangladesh', 'Sri_Lanka', 'Hong_Kong', 'Japan']
# onb = ['SRI_LANKA']
# # country_codes = {
# #     "ID": "Indonesia",
# #     "BD": "Bangladesh",
# #     "LK": "Sri_Lanka",
# #     "HK": "Hong_Kong",
# #     "JP": "Japan"
# # }
# # country_codes.values()

# for c in onb:
#     if c in ORGANIZATION_ID:
#         print(c, '-', ORGANIZATION_ID[c])

In [ ]:
%%time
for ctr_id in tqdm(ORG_ID_R.keys()):
    # ctr_id = ORGANIZATION_ID[ctr]
    ctr = ORG_ID_R[ctr_id]

    ## Offers

    offer = offer_[ctr_id].copy()

    offer.rename(columns={"creation_date": "created_date"}, inplace=True)
    offer.columns = offer.columns.str.lower()
    offer = offer.drop(
        columns=[
            "customer_id",
            "offer_row_id",
            "aag_id",
            "cpa_id",
            "composite_id",
            "bpm_integration_id",
        ]
    ).drop_duplicates()

    offer = offer.add_suffix("_off").rename(
        columns={
            "organization_id_off": "organization_id",
            "offer_id_off": "offer_id",
            "gsfa_customer_id_off": "gsfa_customer_id",
            "opportunity_id_off": "opportunity_id",
        }
    )

    offer["time_taken"] = (
        offer.updated_date_off - offer.created_date_off
    ) / np.timedelta64(1, "D")

    ## Audit

    audit = audit_[ctr_id].copy()

    audit.columns = audit.columns.str.lower()
    audit.drop(columns=["contactrole"], inplace=True)
    audit.drop_duplicates(inplace=True)

    # sorting by updated date
    audit.sort_values(
        ["organization_id", "offer_id", "updated_date", "new_value"], inplace=True
    )

    ## Outcomes

    # outcome from AUDIT - final record in GROUP BY of date-sorted status updates
    outcome = audit.groupby("offer_id").last().reset_index()
    outcome = outcome.merge(
        audit.groupby("offer_id")
        .new_value.value_counts()
        .unstack()["Offer with Customer"]
        .rename("sent_out"),
        how="left",
        on="offer_id",
    )

    # default status
    outcome["outcome"] = "In Progress"
    # Accepted offers
    outcome.loc[outcome.new_value == "Agreement Created", "outcome"] = "Accepted"
    # Rejected offers
    outcome.loc[outcome.new_value == "Customer Rejected", "outcome"] = "Rejected"
    # Invalidated internally
    outcome.loc[outcome.new_value == "Invalidated", "outcome"] = "Invalidated DHL"
    # Invalidated after offering to customer
    outcome.loc[
        (outcome.new_value == "Invalidated")
        & (outcome.old_value == "Customer Rejected"),
        "outcome",
    ] = "Rejected"
    outcome.loc[
        (outcome.new_value == "Invalidated")
        & (outcome.old_value == "Customer Accepted"),
        "outcome",
    ] = "Rejected"
    outcome.loc[
        (outcome.new_value == "Invalidated")
        & (outcome.old_value == "Offer with Customer"),
        "outcome",
    ] = "Rejected"

    outcome = outcome[outcome.sent_out.notna()]
    outcome.reset_index(drop=True, inplace=True)

    ## Opportunity

    oppo = oppo_[ctr_id].copy()

    oppo.columns = oppo.columns.str.lower()
    oppo.drop_duplicates(inplace=True)
    oppo = oppo.add_suffix("_oppo").rename(
        columns={
            "organization_id_oppo": "organization_id",
            "opportunity_id_oppo": "opportunity_id",
            "gsfa_customer_id_oppo": "gsfa_customer_id",
        }
    )

    ## Customer

    customer = customer_[ctr_id].copy()

    customer.drop_duplicates(inplace=True)
    customer.columns = customer.columns.str.lower()
    customer = customer.add_suffix("_cust").rename(
        columns={
            "organization_id_cust": "organization_id",
            "gsfa_customer_id_cust": "gsfa_customer_id",
        }
    )

    ## Shipments

    shipment = shipment_[ctr_id].copy()

    shipment.columns = shipment.columns.str.lower()
    shipment.rename(columns={"last_updated_date": "updated_date"}, inplace=True)
    shipment.published_revenue.replace(0, np.nan, inplace=True)
    shipment["discount"] = (
        100.0 - 100.0 * shipment.expected_revenue / shipment.published_revenue
    )
    shipment.sort_values(
        ["organization_id", "offer_id", "product_cluster"], inplace=True
    )
    shipment.drop_duplicates()
    shipment = shipment.add_suffix("_shp").rename(
        columns={
            "organization_id_shp": "organization_id",
            "offer_id_shp": "offer_id",
            "gsfa_customer_id_shp": "gsfa_customer_id",
        }
    )

    # Merging

    # start from SHIPMENT (as a basis for product-level prediction)
    step1 = shipment.merge(
        offer, how="left", on=["organization_id", "offer_id", "gsfa_customer_id"]
    )

    step2 = step1.merge(
        oppo, how="left", on=["organization_id", "opportunity_id", "gsfa_customer_id"]
    )

    step3 = step2.merge(
        customer.drop(columns=["customer_name_cust"]),
        how="left",
        on=["organization_id", "gsfa_customer_id"],
    )
    step4 = step3.merge(
        outcome[["offer_id", "outcome", "sent_out", "new_value", "old_value"]],
        how="left",
        on="offer_id",
    )

    # Initial filtering & enrichment

    # merge into final dataset
    final = step4[
        (step4.sent_out.notna()) & (step4.outcome != "Invalidated DHL")
    ].copy()
    final.shape

    # Define product domains - TDI, DDI and DOM
    final["product_mix"] = final.product_cluster_shp.map(
        {
            "TDExport": "PROD.MIX:TDEXPORT",
            "TDImport": "PROD.MIX:TDIMPORT",
            "TD3rd": "PROD.MIX:TD3RD",
            "DDExport": "PROD.MIX:DDEXPORT",
            "DDImport": "PROD.MIX:DDIMPORT",
            "DD3rd": "PROD.MIX:DD3RD",
            "TDDom": "PROD.MIX:TDDOM",
            "TD3rdDom": "PROD.MIX:TD3DOM",
        }
    ).fillna("PROD.MIX:OTHER")

    # Define product domains - TDI, DDI and DOM
    final["product_domain"] = final.product_cluster_shp.map(
        {
            "TDExport": "TDI",
            "TDImport": "TDI",
            "TD3rd": "TDI",
            "TDDom": "DOM",
            "TD3rdDom": "DOM",
            "DDExport": "DDI",
            "DDImport": "DDI",
            "DD3rd": "DDI",
        }
    ).fillna("OTHER")

    # integrity flags
    final["empty_exp"] = (final.expected_revenue_shp.isna()).astype("int")
    final["empty_pub"] = (final.published_revenue_shp.isna()).astype("int")
    final["empty_any"] = final.empty_exp | final.empty_pub
    final["empty_all"] = final.empty_exp & final.empty_pub

    final["valid_discount"] = (
        (final.discount_shp >= 0) & (final.discount_shp <= 100)
    ).astype("int")

    ## Keep only products (shipments) with valid discount

    final = final[final.valid_discount == 1]

    ## Currency adjustment for dual-currency countries (Domestic vs International)

    if ctr in dual_org_country_names:
        print("dual currency adjustment...")
        er = ctr2ex_rate_dom2int[ctr]
        cond = final.product_domain == "DOM"
        for col in [
            "published_revenue_shp",
            "recommended_revenue_shp",
            "expected_revenue_shp",
        ]:
            final.loc[cond, col] = final[cond][col] / er
        # cond = final.product_domain == 'TDI'
        # final.loc[cond, 'recommended_revenue_shp'] = final[cond].recommended_revenue_shp / er

    ## Offer-level revenues, discounts, and offer composition (by product cluster & domain)

    final["offer_expected_revenue"] = final.groupby(
        "offer_id"
    ).expected_revenue_shp.transform("sum")
    final["offer_published_revenue"] = final.groupby(
        "offer_id"
    ).published_revenue_shp.transform("sum")
    final["offer_recommended_revenue"] = final.groupby(
        "offer_id"
    ).recommended_revenue_shp.transform("sum")

    final["log_offer_published_revenue"] = final.offer_published_revenue.apply(np.log)
    final["offer_discount"] = 100.0 * (
        1.0 - final.offer_expected_revenue / final.offer_published_revenue
    )
    final["offer_products"] = final.groupby("offer_id").product_cluster_shp.transform(
        "nunique"
    )

    # for each offer calcualte the breakdown into product domains (% of products in total products)
    aaa = (
        final.groupby(["offer_id", "product_domain"])
        .published_revenue_shp.sum()
        .unstack()
    )
    bbb = aaa.div(aaa.sum(axis=1), axis=0).reset_index().fillna(0)
    bbb.rename(
        columns={
            "TDI": "fraction_tdi",
            "DDI": "fraction_ddi",
            "DOM": "fraction_dom",
            "OTHER": "fraction_other",
        },
        inplace=True,
    )
    final = final.merge(bbb, on="offer_id", how="left")

    # for each offer calcualte the breakdown into product mix (% of products in total products)
    aaa = (
        final.groupby(["offer_id", "product_mix"]).published_revenue_shp.sum().unstack()
    )
    bbb = aaa.div(aaa.sum(axis=1), axis=0).reset_index().fillna(0)
    final = final.merge(bbb, on="offer_id", how="left")

    ## Transformations & enrichments

    # - Derive binary variables (e-commerce)
    # - Add log-transformations on certain numeric variables
    # - etc.

    # - log transforms of weight variables
    final.shipments_shp.replace(0, np.nan, inplace=True)
    final.wps_shp.replace(0, np.nan, inplace=True)
    final["weight"] = final.shipments_shp * final.wps_shp

    final["log_shipments"] = final.shipments_shp.apply(np.log)
    final["log_wps"] = final.wps_shp.apply(np.log)
    final["log_weight"] = final.weight.apply(np.log)

    final["log_discount"] = final.discount_shp.apply(np.log)

    final.published_revenue_shp.replace(0, np.nan, inplace=True)
    final.new_potential_revenue_oppo.replace(0, np.nan, inplace=True)
    final.opportunity_tpr_oppo.replace(0, np.nan, inplace=True)

    final["log_published_revenue"] = final.published_revenue_shp.apply(np.log)
    final["log_published_revenue_sq"] = (
        final.log_published_revenue * final.log_published_revenue
    )

    final["log_potential_revenue"] = (
        final.new_potential_revenue_oppo.apply(np.log)
        if final.new_potential_revenue_oppo.notna().sum() > 0
        else np.nan
    )
    final["log_potential_revenue_tpr"] = (
        final.opportunity_tpr_oppo.apply(np.log)
        if final.opportunity_tpr_oppo.notna().sum() > 0
        else np.nan
    )
    diff_ = (
        final["log_potential_revenue"].fillna(-1)
        != final["log_potential_revenue"].fillna(-1)
    ).sum()
    print(f"{diff_}/{len(final)} = {diff_ / len(final):.2f}")

    # E-commerce flag
    final["ecomm"] = 0
    final.loc[final.physical_channel_cust.str.lower() == "e-commerce", "ecomm"] = 1

    # Change added after ecom definition change
    final.loc[final.physical_channel_cust.str.lower() == "b2c e-commerce", "ecomm"] = 1
    final.loc[final.physical_channel_cust.str.lower() == "b2b e-commerce", "ecomm"] = 1

    ## Mapped categoricals

    # Pre-defined mapping

    final["lead_oppo"] = final.reason_for_lead_oppo.replace(LEAD_MAPPING)
    final.loc[
        final.lead_oppo.notna() & ~final.lead_oppo.isin(set(LEAD_MAPPING.values())),
        "lead_oppo",
    ] = "LEAD:OTHER"

    final["lead_cust"] = final.reason_for_lead_cust.replace(LEAD_MAPPING)
    final.loc[
        final.lead_cust.notna() & ~final.lead_cust.isin(set(LEAD_MAPPING.values())),
        "lead_cust",
    ] = "LEAD:OTHER"

    final["initiator_oppo"] = final.lead_source_type_oppo.replace(INITIATOR_MAPPING)
    final.loc[
        final.initiator_oppo.notna()
        & ~final.initiator_oppo.isin(set(INITIATOR_MAPPING.values())),
        "initiator_oppo",
    ] = "INIT:?"

    final["initiator_cust"] = final.lead_source_type_cust.replace(INITIATOR_MAPPING)
    final.loc[
        final.initiator_cust.notna()
        & ~final.initiator_cust.isin(set(INITIATOR_MAPPING.values())),
        "initiator_cust",
    ] = "INIT:?"

    final["competitor_oppo"] = final.primary_competitor_oppo.replace(COMPETITOR_MAPPING)
    final.loc[
        final.competitor_oppo.notna()
        & ~final.competitor_oppo.isin(set(COMPETITOR_MAPPING.values())),
        "competitor_oppo",
    ] = "COMP:OTHER"

    final["competitor_cust"] = final.competitor_cust.replace(COMPETITOR_MAPPING)
    final.loc[
        final.competitor_cust.notna()
        & ~final.competitor_cust.isin(set(COMPETITOR_MAPPING.values())),
        "competitor_cust",
    ] = "COMP:OTHER"

    final["industry"] = "IND:" + final.industry_code_cust.fillna("NODATA").apply(
        lambda x: x[:4]
    )
    final.industry.replace("IND:NODA", np.nan, inplace=True)

    # Final Filtering

    ## Keep only offers without OTHER domain

    # clusters - only include TDI, DDI and DOM
    dataset = (
        final[final.fraction_other == 0].copy()
        if "fraction_other" in final.columns
        else final.copy()
    )

    ## Remove offers with CustomPIDs and 100% discounts

    # CustomPID with zero expected revenue
    cpid_list = dataset[dataset.selected_pid_name_shp == "CustomPID"].offer_id.to_list()
    dataset = dataset[
        ~(dataset.offer_id.isin(cpid_list) & (dataset.expected_revenue_shp == 0))
    ]

    ## Establish order within an opportunity, and set the flag if the offer is final (within its OPPO)
    # * order

    # add order (X out of Y) on the clean set
    order_in_oppo = (
        dataset.groupby(["opportunity_id", "offer_id"])
        .created_date_off.first()
        .reset_index()
        .sort_values("created_date_off")
    )
    order_in_oppo["order_in_oppo"] = (
        order_in_oppo.groupby("opportunity_id").cumcount() + 1
    )

    # calculate number of offers
    no_of_offers = dataset.groupby("opportunity_id").offer_id.nunique().reset_index()
    no_of_offers.rename(columns={"offer_id": "offers_in_oppo"}, inplace=True)

    dataset = dataset.merge(
        order_in_oppo.drop(columns="created_date_off"),
        how="left",
        on=["offer_id", "opportunity_id"],
    )
    dataset = dataset.merge(no_of_offers, how="left", on="opportunity_id")

    # sequence and status helper fields
    dataset["finality"] = "final"
    dataset.loc[dataset.offers_in_oppo != dataset.order_in_oppo, "finality"] = "interim"

    # final status & discount
    a = (
        dataset.sort_values("created_date_off")
        .groupby(["opportunity_id", "product_cluster_shp"])[["outcome", "discount_shp"]]
        .last()
        .reset_index()
        .rename(
            columns={"outcome": "final_outcome", "discount_shp": "final_discount_shp"}
        )
    )
    a.head(3)

    dataset = dataset.merge(a, how="left", on=["opportunity_id", "product_cluster_shp"])

    ## Define revenue bins

    # New approach
    # Approved by Alex Goldirev to replace manual limits with data-guided limits.
    # We discard top 5% of offers (by offer revenue)

    labels = ["low", "mid-low", "medium", "mid-high", "high"]

    dataset["revenue_bin"] = np.nan

    for country in dataset.organization_id.unique():
        # offer revenues
        offer_revenues = dataset[dataset.organization_id == country][
            ["offer_id", "offer_published_revenue"]
        ].drop_duplicates()

        dataset.loc[dataset.organization_id == country, "revenue_bin"] = pd.cut(
            dataset[dataset.organization_id == country].offer_published_revenue,
            np.linspace(0, offer_revenues.offer_published_revenue.quantile(0.95), 6),
            right=False,
            labels=labels,
            precision=3,
            duplicates="drop",
        )

    dataset["discount_oms"] = (
        100.0 - 100.0 * dataset.recommended_revenue_shp / dataset.published_revenue_shp
    )
    dataset["discount_delta"] = dataset.discount_shp - dataset.discount_oms

    # Save `dataset_short.csv`

    dataset = dataset.reset_index(drop=True)
    path_ = os.path.join(countries_dir, f"{ctr}.csv")
    dataset.to_csv(path_, index=False)

    print(f"{ctr.ljust(20)} - {len(dataset):8,} | saved to: {path_}")
    counts["organization_id"].append(ctr_id)
    counts["country"].append(ctr)
    counts["n_offers"].append(len(dataset))

In [ ]:
for ctr in ORGANIZATION_ID:
    final = pd.read_csv(f"{countries_dir}/{ctr}.csv", low_memory=False)
    final = final[
        final.log_potential_revenue.notna() | final.log_potential_revenue_tpr.notna()
    ]
    diff_ = (
        final["log_potential_revenue"].fillna(-1)
        != final["log_potential_revenue_tpr"].fillna(-1)
    ).sum()
    print(
        f"{ctr.ljust(26, '.')} {diff_:4}/{len(final):6} = {diff_ / len(final) * 100:.1f}%"
    )

In [ ]:
a = []
s = 0
for ctr in ORGANIZATION_ID:
    final = pd.read_csv(f"{countries_dir}/{ctr}.csv", low_memory=False)

    final["physical_channel_cust2"] = final.physical_channel_cust.map(
        lambda x: x if x in ["B2C E-Commerce", "B2B E-Commerce"] else np.nan
    )
    final.physical_channel_cust = final.physical_channel_cust.fillna("~")
    final.physical_channel_cust2 = final.physical_channel_cust2.fillna("~")

    diff_ = (final["physical_channel_cust"] != final["physical_channel_cust2"]).sum()
    print(
        f"{ctr.ljust(26, '.')} {diff_:4}/{len(final):6} = {diff_ / len(final) * 100:.1f}%"
    )
    # display(final.industry.value_counts(dropna=False))
    x = final.industry.fillna("nan").value_counts()
    s += x
    a.append(final[["industry"]])

    # display(s)
    # break
# final

In [ ]:
df = pd.concat(a)
(df.value_counts(dropna=False) / len(df) * 100).round(1)
# df

In [ ]:
df2 = pd.read_csv("industry_counts.csv")
df2.columns = ["ind", "n"]
df2["n"] = df2["n"] / df2["n"].sum() * 100
df2.set_index("ind").n.round(1).sort_values(ascending=False).head(10)

In [85]:
final["physical_channel_cust2"] = final.physical_channel_cust.map(
    lambda x: x if x in ["B2C E-Commerce", "B2B E-Commerce"] else np.nan
)

final.physical_channel_cust = final.physical_channel_cust.fillna("~")
final.physical_channel_cust2 = final.physical_channel_cust2.fillna("~")

In [ ]:
final

In [ ]:
# df = pd.read_excel("_data\Countries_dataset_transformed.xlsx")
# df = pd.read_csv("_data\Countries_dataset_transformed.csv")
df.head()

In [ ]:
df

In [ ]:
final = (
    df[df.organization_id == ORGANIZATION_ID["AUSTRIA"]]
    .replace("?", np.nan)
    .replace(0, np.nan)
)
final

In [15]:
final0 = final.copy()

In [ ]:
final0[final0.product_domain != "OTHER"].groupby("product_domain").head(4)[
    [
        "product_domain",
        "published_revenue_shp",
        "recommended_revenue_shp",
        "expected_revenue_shp",
    ]
].sort_values("product_domain")

In [17]:
if ctr in dual_org_country_names:
    er = ctr2ex_rate_dom2int[ctr]
    print(er)
    cond = final.product_domain == "DOM"
    for col in [
        "published_revenue_shp",
        "recommended_revenue_shp",
        "expected_revenue_shp",
    ]:
        final.loc[cond, col] = final[cond][col] / er

    cond = final.product_domain == "TDI"
    final.loc[cond, "recommended_revenue_shp"] = (
        final[cond].recommended_revenue_shp / er
    )

In [ ]:
final[final.product_domain != "OTHER"].groupby("product_domain").head(4)[
    [
        "product_domain",
        "published_revenue_shp",
        "recommended_revenue_shp",
        "expected_revenue_shp",
    ]
].sort_values("product_domain")

In [19]:
# (dataset.updated_date_shp < pd.to_datetime('2024-01-02')).sum()

In [ ]:
counts_df = pd.DataFrame(counts).drop_duplicates()
path_ = os.path.join(countries_dir, "0_counts.csv")
counts_df.to_csv(path_, index=False)
print(f"saved to: {path_}")
display(counts_df.sort_values("n_offers", ascending=False))